In [ ]:
import pandas as pd
import panel as pn
import holoviews as hv

from earthsim.xmsmesh import GenerateMesh, ViewMesh

hv.extension('bokeh')

In [ ]:
sample_poly1=dict(
    Longitude = [-10114986, -10123906, -10130333, -10121522, -10129889, -10122959],
    Latitude  = [  3806790,   3812413,   3807530,   3805407,   3798394,   3796693])
sample_poly2=dict(
    Longitude = [-10095762, -10101582, -10104906],
    Latitude  = [  3804189,   3817180,   3799659])
sample_points = pd.DataFrame({
    'Longitude': [-10100750],
    'Latitude':  [ 3807009.333],
    'Size':      [   100]})

In [ ]:
# construct the stages of the pipeline
stages = [
    ('Generate Mesh', GenerateMesh(polys=[sample_poly1, sample_poly2], points=sample_points)),
    ('View Mesh', ViewMesh)
]

# construct the pipeline
pipeline = pn.pipeline.Pipeline(stages)

pipeline.layout  # display in the cell below
# pipeline.layout.show()  # launch in separate browser window